# Machine Learning 1 - Aprendizado Não Supervisionado  
Prof: Nielsen Rechia

### Autores: 
Roberto Coutinho  
Thais Galho

### Exercício censo 1995 nos Estados Unidos

* O conjunto de dados ```adult``` trata de um censo demográfico realizado nos Estados Unidos em 1995
* O atributo classe se refere a caso um indivíduo ganha mais que 50,000 dólares por ano ou não

Tarefas:

* Separe o dataset em treino e teste (use seed=0), com 75% para treino e 25% para teste
* Adicione valores faltantes, trate os atributos categóricos
* Treine uma [árvore de decisão](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) neste conjunto
* Reporte a [acurácia](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score)
* Visualize a árvore de decisão (tutorial: [link](http://scikit-learn.org/stable/modules/tree.html#classification))

Descrição dos arquivos:

* adult.csv: dataset
* adult_description: descrição do conjunto de dados

### Imports

In [2]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import random
from sklearn.preprocessing import LabelEncoder
from IPython.display import display



### Carrega conjunto de dados, define tipos de colunas

In [10]:

random.seed(0)
np.random.seed(0)  # garante que o conjunto de dados seja sempre particionado da mesma forma

df = pd.read_csv(
    filepath_or_buffer='censo/adult.data', 
    dtype={
        'age': np.float32,
        'workclass': 'category',
        'fnlwgt': np.float32,
        'education': 'category',
        'education-num': np.float32,
        'marital-status': 'category',
        'occupation': 'category',
        'relationship': 'category',
        'race': 'category',
        'sex': 'category',
        'capital-gain': np.float32,
        'capital-loss': np.float32,
        'hours-per-week': np.float32,
        'native-country': 'category',
        'salary': 'category',
    },
    na_values='?',  # detecta ? como um valor NaN
    skipinitialspace=True  # ignora espaços em branco entre valores
)

columns= ["age", 
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
        "class"]

df.columns = columns

df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


## Pré-processamento
### Substitui valores faltantes pela média de cada atributo

In [29]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0)

imputer.fit_transform(df.iloc[:, [4,10,11, 12]])

array([[1.3000e+01, 0.0000e+00, 0.0000e+00, 1.3000e+01],
       [9.0000e+00, 0.0000e+00, 0.0000e+00, 4.0000e+01],
       [7.0000e+00, 0.0000e+00, 0.0000e+00, 4.0000e+01],
       ...,
       [9.0000e+00, 0.0000e+00, 0.0000e+00, 4.0000e+01],
       [9.0000e+00, 0.0000e+00, 0.0000e+00, 2.0000e+01],
       [9.0000e+00, 1.5024e+04, 0.0000e+00, 4.0000e+01]])

In [33]:
df.head()
#df.isna().sum()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


### Transforma atributos preditivos categóricos em numéricos 
**seja binarizando ou atribuindo um número a cada valor**

In [47]:
df['fnlwgt'] = df['fnlwgt'].astype('category')
df['workclass'] = df['workclass'].astype('category')
df['education'] = df['education'].astype('category')
df['marital-status'] = df['marital-status'].astype('category')
df['occupation'] = df['occupation'].astype('category')
df['relationship'] = df['relationship'].astype('category')
df['race'] = df['race'].astype('category')
df['sex'] = df['sex'].astype('category')
df['native-country'] = df['native-country'].astype('category')
categories = df.select_dtypes(['category']).columns
df[categories] = df[categories].apply(lambda x:x.cat.codes) 
df.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,50,6,2925,9,13,2,4,0,4,1,0,0,13,39,0
1,38,4,14085,11,9,0,6,1,4,1,0,0,40,39,0
2,53,4,15335,1,7,2,6,0,2,1,0,0,40,39,0
3,28,4,19354,9,13,2,10,5,2,0,0,0,40,5,0
4,37,4,17699,12,14,2,4,5,4,0,0,0,40,39,0


### Transforma o atributo classe (que é binário) em numérico ordinal

**Atributos classe não precisam ser binarizados.**

Esse código também reenvia o atributo classe para a última posição no dataset, já que ele foi deslocado.

In [48]:
fact, class_labels = pd.factorize(df['class'].astype(np.object))

df['class'] = fact
columns = df.columns.tolist()
columns.pop(columns.index('class'))
columns.append('class')
df = df.reindex(columns=columns)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,50,6,2925,9,13,2,4,0,4,1,0,0,13,39,0
1,38,4,14085,11,9,0,6,1,4,1,0,0,40,39,0
2,53,4,15335,1,7,2,6,0,2,1,0,0,40,39,0
3,28,4,19354,9,13,2,10,5,2,0,0,0,40,5,0
4,37,4,17699,12,14,2,4,5,4,0,0,0,40,39,0


#### Separa dados entre treino e teste

In [49]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = df[df.columns[:-1]], df[df.columns[-1]]

# utiliza 25% do dataset para teste
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, stratify=y)  

In [56]:
X_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
29136,29,4,12837,12,14,4,10,1,4,1,0,0,55,34
30039,22,4,12463,9,13,4,12,1,4,0,0,0,20,39
31277,49,4,5764,8,11,2,12,0,4,1,0,0,25,39
31324,25,2,6794,7,12,4,1,3,2,0,0,0,38,39
29955,68,0,4579,15,10,2,0,5,4,0,0,0,12,39


In [57]:
Y_train.head()

29136    0
30039    0
31277    0
31324    0
29955    1
Name: class, dtype: int64

## Treinamento
#### Treina uma árvore de decisão, testa no conjunto de testes e calcula a acurácia

In [62]:
dt = DecisionTreeClassifier(random_state=0)

dt.fit(X_train, Y_train)

predictions = dt.predict(X_test)

## Avaliação de desempenho

#### Define o método para calcular a acurácia do modelo

In [69]:
def getAccuracy(testset, predictions):
    correct = 0
    for id_test, test in enumerate(testset):
        if test == predictions[id_test]:
            correct += 1
    return (correct / float(len(testset))) * 100.0

print('accuracy score:', getAccuracy(Y_test, predictions))

('accuracy score:', 80.85995085995086)


#### Matriz de confusão

In [88]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(Y_test, predictions)
print 'Matriz de confusão'
print cm
print
tn = float(cm[0][0])
fp = float(cm[0][1])
fn = float(cm[1][0])
tp = float(cm[1][1])
actual_yes = fn+tp
actual_no = tn+fp
predicted_yes = fp+tp
predicted_no = tn+fn
total = float(len(df))
print 'Total : '+ str(total)

accuracy_2 = round((tp+tn)/total,3) # Overall, how often is the classifier correct?
print 'Acurácia : ' + str(accuracy_2)

misclassification_rate = round((fp+fn)/total,3) # Overall, how often is it wrong?
print 'True positives : ' +str(misclassification_rate
print
true_positive = round(tp/actual_yes,3) # When it's actually yes, how often does it predict yes?
print 'True positives : ' +str(true_positive)

false_positive = round(fp/actual_no,3) # When it's actually no, how often does it predict yes?
print 'False positives : ' +str(false_positive)

specificity = round(tn/actual_no,3) # When it's actually no, how often does it predict no?
print 'Specificity : ' +str(specificity)

precision = round(tp/predicted_yes,3) # When it predicts yes, how often is it correct?
print 'Precision : ' +str(precision)

prevalence = round(actual_yes/total,3) # How often does the yes condition actually occur in our sample?
print prevalence

f1 = round(2 * ((precision * true_positive) / (precision + true_positive)),3)
print 'F1 Score : ' +str(f1)

Matriz de confusão
[[5358  822]
 [ 736 1224]]

Total : 32560.0
Acurácia : 0.202
0.048

True positives : 0.624
False positives : 0.133
Specificity : 0.867
Precision : 0.598
0.06
F1 Score : 0.611


### Bônus: visualizando a árvore


In [74]:
import graphviz 
import pydotplus
from IPython.display import Image 

dot_data = tree.export_graphviz(
    dt, out_file=None, 
    feature_names=df.columns[:-1],  # ignora classe
    class_names=class_labels,  
    filled=True, rounded=True,  
    special_characters=True
)  

graph = pydotplus.graphviz.graph_from_dot_data(dot_data)
Image(graph.create_png())

ImportError: No module named graphviz